<a href="https://colab.research.google.com/github/LesterYe/BAGLS-Image-Segmentation/blob/main/model_resnet50_BAGLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec  5 04:37:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

!pip install wandb --upgrade
import wandb
from wandb.keras import WandbCallback

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 28.2 MB/s 
     |████████████████████████████████| 168 kB 75.0 MB/s 
     |████████████████████████████████| 182 kB 68.8 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 80.6 MB/s 
     |████████████████████████████████| 166 kB 77.8 MB/s 
     |████████████████████████████████| 166 kB 83.1 MB/s 
     |████████████████████████████████| 162 kB 82.9 MB/s 
     |████████████████████████████████| 162 kB 75.7 MB/s 
     |████████████████████████████████| 158 kB 82.3 MB/s 
     |████████████████████████████████| 157 kB 80.4 MB/s 
     |████████████████████████████████| 157 kB 82.1 MB/s 
     |████████████████████████████████| 157 kB 80.9 MB/s 
     |████████████████████████████████| 157 kB 82.2 MB/s 
     |████████████████████████████████| 157 kB 80.3 MB/s 
     |███████████████████████████

In [6]:
key = '1752d2f0785658c328f0ed6d6c0bc7debd495d9c'
wandb.login(key=key)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
root = '/content'
path = '/content/drive/MyDrive/BAGLS-Image-Segmentation/'
dataset1zip_path = os.path.join(path,'dataset1.zip')
dataset1_path = os.path.join(root,'dataset1')

os.chdir(root)
os.getcwd()

'/content'

In [5]:
!cp '{dataset1zip_path}' .
!unzip -q dataset1.zip
!rm dataset1.zip

## ResNet50

In [14]:
lr = 1e-4
train_batchsize = 32

In [15]:
configs = dict(
    data_flag = 'BAGLS_Abnormal_Normal',
    image_width = 336,
    image_height = 336,
    batch_size = train_batchsize,
    model_name = 'resnet50',
    pretrain_weights = 'imagenet',
    epochs = 100,
    init_learning_rate = lr,
    optimizer = 'RMSProp',
    loss_fn = 'categorical_crossentropy',
    metrics = ['acc'],
    earlystopping_patience = 2
)

In [22]:
## Using wandb for MLOps
project = 'endoscopy_BAGLS'

wandb.init(project=project, config=configs)

# # values for hyperparamters
# config = wandb.config
# config.learning_rate = lr
# config.batch_size = train_batchsize

In [23]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, Sequential, save_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

model = Sequential()
resNet = ResNet50(input_shape = (512,256,3), weights='imagenet', include_top=False)
model.add(resNet)
model.add(Flatten())
model.add(Dense(units=250, activation='relu')) #units=250
# model.add(Dropout(0.5))
model.add(Dense(units=2, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 8, 2048)       23587712  
                                                                 
 flatten_2 (Flatten)         (None, 262144)            0         
                                                                 
 dense_4 (Dense)             (None, 250)               65536250  
                                                                 
 dense_5 (Dense)             (None, 2)                 502       
                                                                 
Total params: 89,124,464
Trainable params: 89,071,344
Non-trainable params: 53,120
_________________________________________________________________


In [24]:
# option to freeze layers
for layer in resNet.layers[:]:
    layer.trainable = False
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 8, 2048)       23587712  
                                                                 
 flatten_2 (Flatten)         (None, 262144)            0         
                                                                 
 dense_4 (Dense)             (None, 250)               65536250  
                                                                 
 dense_5 (Dense)             (None, 2)                 502       
                                                                 
Total params: 89,124,464
Trainable params: 65,536,752
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [25]:
from zmq.sugar.socket import T
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Change the batchsize according to your system RAM
data_path = dataset1_path
train_batchsize = 32
val_batchsize = 32
train_dir = data_path
val_dir = data_path

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True,
    validation_split = 0.2 # set train-val split ratio
) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (512,256),
    batch_size = train_batchsize,
    class_mode = 'categorical',
    subset = 'training' # set as train data
)

val_generator = train_datagen.flow_from_directory(
    train_dir, # same dir as train data
    target_size = (512,256),
    batch_size = val_batchsize,
    class_mode = 'categorical',
    subset = 'validation' # set as val data
)

Found 42120 images belonging to 2 classes.
Found 10530 images belonging to 2 classes.


In [26]:
callbacks = [
    WandbCallback(),
    # EarlyStopping(patience=2),
    # ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    # TensorBoard(log_dir='./logs')
]

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = optimizers.RMSprop(learning_rate=lr),
    metrics = ['acc']
)

# Train model
hist = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples / train_generator.batch_size,
    epochs = 100,
    callbacks = callbacks,
    validation_data = val_generator,
    validation_steps = val_generator.samples / val_generator.batch_size,
    verbose = 1
)

# # Save model
# model.save('resNet50-trainlast4.hd5')

Epoch 1/100
1317/1316 [==============================] - ETA: 0s - loss: 0.7655 - acc: 0.6044

wandb: Adding directory to artifact (/content/wandb/run-20221204_194347-k8lu64ot/files/model-best)... Done. 3.7s


1316/1316 [==============================] - 482s 364ms/step - loss: 0.7655 - acc: 0.6044 - val_loss: 0.6294 - val_acc: 0.6794
Epoch 2/100
1317/1316 [==============================] - ETA: 0s - loss: 0.6571 - acc: 0.6371

wandb: Adding directory to artifact (/content/wandb/run-20221204_194347-k8lu64ot/files/model-best)... Done. 3.6s


1316/1316 [==============================] - 473s 359ms/step - loss: 0.6571 - acc: 0.6371 - val_loss: 0.6021 - val_acc: 0.6973
Epoch 3/100
1317/1316 [==============================] - ETA: 0s - loss: 0.6293 - acc: 0.6603

...

1316/1316 [==============================] - 472s 358ms/step - loss: 0.5516 - acc: 0.7169 - val_loss: 0.5263 - val_acc: 0.7417
Epoch 18/100
1316/1316 [==============================] - 442s 336ms/step - loss: 0.5472 - acc: 0.7190 - val_loss: 0.5368 - val_acc: 0.7449
Epoch 19/100
1316/1316 [==============================] - 442s 336ms/step - loss: 0.5437 - acc: 0.7212 - val_loss: 0.5800 - val_acc: 0.6928
Epoch 20/100
1316/1316 [==============================] - 443s 336ms/step - loss: 0.5422 - acc: 0.7216 - val_loss: 0.5476 - val_acc: 0.7381
Epoch 21/100
1316/1316 [==============================] - 442s 336ms/step - loss: 0.5368 - acc: 0.7287 - val_loss: 0.5272 - val_acc: 0.7207
Epoch 22/100
1317/1316 [==============================] - ETA: 0s - loss: 0.5351 - acc: 0.7236

wandb: Adding directory to artifact (/content/wandb/run-20221204_194347-k8lu64ot/files/model-best)... Done. 3.7s


1316/1316 [==============================] - 472s 359ms/step - loss: 0.5351 - acc: 0.7236 - val_loss: 0.5227 - val_acc: 0.7218
Epoch 23/100
1316/1316 [==============================] - 443s 337ms/step - loss: 0.5335 - acc: 0.7270 - val_loss: 1.1085 - val_acc: 0.4187
Epoch 24/100
1317/1316 [==============================] - ETA: 0s - loss: 0.5285 - acc: 0.7316

wandb: Adding directory to artifact (/content/wandb/run-20221204_194347-k8lu64ot/files/model-best)... Done. 3.7s


1316/1316 [==============================] - 471s 358ms/step - loss: 0.5285 - acc: 0.7316 - val_loss: 0.4952 - val_acc: 0.7533
Epoch 25/100
1316/1316 [==============================] - 442s 336ms/step - loss: 0.5249 - acc: 0.7315 - val_loss: 0.4995 - val_acc: 0.7515
Epoch 26/100
1316/1316 [==============================] - 444s 337ms/step - loss: 0.5235 - acc: 0.7334 - val_loss: 0.5048 - val_acc: 0.7586
Epoch 27/100
1119/1316 [========================>.....] - ETA: 54s - loss: 0.5180 - acc: 0.7332

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])